In [1]:
import pandas as pd

df = pd.DataFrame()

In [2]:
import os

In [3]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

75

In [4]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import re

In [5]:
for a in tqdm(range(len(list_file)), desc='Progress', ncols=77):
    with open('./part/' + list_file[a], 'r', encoding='UTF-8') as file:
        resp = file.read()
        
    # = = = = = = = = = = = = = = =
    
    soup = BeautifulSoup(resp, 'lxml')
    html = etree.HTML(str(soup).encode('ascii', 'xmlcharrefreplace').decode('utf-8'))
    
    # = = = = = = = = = = = = = = =
    
    title = html.xpath('//h1[@id="title"]/span/text()')[0].strip()

    # = = = = = = = = = = = = = = =

    price = html.xpath('//span[@class="a-offscreen"]/text()')[1].strip()
    
    # = = = = = = = = = = = = = = =
    
    about = ''
    list_about = html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')
    for i in range(len(list_about)):
        about += str(i+1) + '. ' + list_about[i].strip() + '\n'

    # = = = = = = = = = = = = = = =
    
    src = html.xpath('//div[@id="imgTagWrapperId"]/img/@src')[0].strip()
    
    # = = = = = = = = = = = = = = =
    
    list_tr = html.xpath('//table[@class="a-normal a-spacing-micro"]/tr')

    # = = = = = = = = = = = = = = =
    
    list_tr_2 = html.xpath('//table[@id="productDetails_techSpec_section_1"]/tr')
            
    # = = = = = = = = = = = = = = =
    
    list_tr_3= html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tr')

    # = = = = = = = = = = = = = = =
    
    df_temp = pd.DataFrame([{'序号': int(list_file[a].replace('.txt', '')),
                             'ASIN': '',
                             'Title': title,
                             'Price': price,
                             'About': about.strip(),
                             'Pic': '',
                             'Url': '',
                             'Src': src}])

    for tr in list_tr:
        df_temp.loc[0, tr.xpath('./td/span/text()')[0].strip().upper()] = tr.xpath('./td/span/text()')[1].strip()

    for tr in list_tr_2:
        df_temp.loc[0, tr.xpath('./th/text()')[0].strip().lower()] = tr.xpath('./td/text()')[0].strip()

    for tr in list_tr_3:
        if tr.xpath('./th/text()')[0].strip() == 'Best Sellers Rank':
            df_temp.loc[0, tr.xpath('./th/text()')[0].strip().upper()] = ' '.join(tr.xpath('./td/descendant::*/text()')).strip()
        else:
            df_temp.loc[0, tr.xpath('./th/text()')[0].strip().upper()] = ' '.join(tr.xpath('./td/text()')).strip()


    df = pd.concat([df, df_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =
    
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|██████████████████████████████| 75/75 [00:16<00:00,  4.61it/s]


,序号,ASIN,Title,Price,About,Pic,Url,Src,BRAND,ORIENTATION,...,country of origin,FINISH TYPE,MOUNTING TYPE,UNIT COUNT,special features,finish,mounting type,included components,SPEED,SEAT WIDTH
0,1,B08HS5XZTR,labwork 3rd Row Power Fold Seat Hinge Motor 6L...,$212.59,"1. When replacing, be aware of position of all...",,,https://m.media-amazon.com/images/I/71nNZhwKqo...,labwork,Front,...,,,,,,,,,,
1,2,B009TKMV2C,Motorcraft MM-990 Seat Actuator Motor,$44.48,1. Country of Origin: United States\n2. Fits m...,,,https://m.media-amazon.com/images/I/71UuRpB5X2...,Motorcraft,,...,‎USA,,,,,,,,,
2,3,B0BDW1KD6R,for Jeep Cherokee Seat Height Adjuster Motor 5...,$29.99,1. Power Seat Height Up and Down Adjuster Moto...,,,https://m.media-amazon.com/images/I/61MzomLgwZ...,Generic,Front and Rear,...,,,,,,,,,,
3,4,B000EQ8XM2,Motorcraft MM949 Power Seat Motor,$89.86,1. Resists abrasion and corrosion\n2. Maximize...,,,https://m.media-amazon.com/images/I/81MKpKYGQe...,,,...,‎China,,,,,,,,,
4,5,B08MZYRNXH,HERCOO Power Seat Track Rail Drive Gear Frame ...,$10.99,"1. FITMENT: Compatible with FX35 2004-2006, FX...",,,https://m.media-amazon.com/images/I/61qMEV2dYt...,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,B0C4F5D7GQ,"Seat Motor Adjustment Gear for BMW 5 Series, 7...",$44.79,1. Seat Motor Adjustment Gear For BMW 5 Series...,,,https://m.media-amazon.com/images/I/41zLTEkW4f...,,,...,,,,,,,,,,
71,72,B00UE2OENI,Jaguar XK8 1997 to Mid 2000 Seat Motor ECM Com...,$17.97,,,,https://m.media-amazon.com/images/I/61iFGGYg1e...,,,...,,,,,,,,,,
72,73,B000NTWWOU,Ford 6L2Z-14547-AA - MOTOR ASY - SEAT ADJ,$233.31,1. Genuine Ford Part 6L2Z-14547-AA\n2. Enginee...,,,https://m.media-amazon.com/images/I/51iy4FD3cW...,,,...,,,,,,,,,,
73,74,B0BFNMXQKV,Seat Height Adjuster Motor Repair Gear for Cad...,$24.99,1. Power Seat Height Up and Down Adjuster Moto...,,,https://m.media-amazon.com/images/I/61p5PvHZUx...,,,...,,,,,,,,,,
